# Reading 13: Functional Programming

Objectives
In this reading you’ll learn a design pattern for implementing functions that operate on sequences of elements, and you’ll see how treating functions themselves as first-class values that we can pass around and manipulate in our programs is an especially powerful idea.

* Map/filter/reduce
* Functional objects
* Higher-order functions


## First-class functions

Let’s start by reviewing an important Big Idea that you should have already encountered in 6.101 [formerly 6.009]: functions as first-class data values, meaning that they can be stored in variables, passed as arguments to functions, and created dynamically like other values.

For example, Math.sqrt is a reference to an object representing the sqrt function. The type of that object is (x: number) => number.

But you can also assign that function object to another variable if you like, and it still behaves like sqrt:

```ts
const mySquareRoot: (x: number) => number = Math.sqrt;
mySquareRoot(16.0); // returns 4.0
```
The type that we declared on mySquareRoot is a [function type expression](https://www.typescriptlang.org/docs/handbook/2/functions.html). Note that the parameter name x is required! If you write (number) => number, it means “a function with a parameter named number of type any that returns a number“, because omitted types are implicitly any.

You can also pass a reference to the function object as a parameter to another function. You can use functions the same way you would use any other value in TypeScript, like numbers or string references or other object references. In other words, ==functions in TypeScript are first-class, which means they can be treated like any other value in the language: passed as parameters, returned as return values, and stored in variables and data structures==.

Programming languages are full of things that are not first-class. For example, access control is not first-class – you can’t pass public or private as a parameter into a function, or store it in a data structure. The notion of public and private is an aspect of your program that TypeScript offers no way to refer to or manipulate at runtime. Similarly, a while loop or if statement is not first-class. You can’t refer to that piece of code all by itself, or manipulate it at runtime.

In old programming languages, only data was first-class: built-in types (like numbers) and user-defined types. But in modern programming languages, like Python and JavaScript, both data and functions are first-class. First-class functions are a very powerful programming idea. The first practical programming language that used them was Lisp, invented by John McCarthy at MIT. But the idea of programming with functions as first-class values actually predates computers, tracing back to Alonzo Church’s lambda calculus. The lambda calculus used the Greek letter λ to define new functions; this term stuck, and you’ll see it as a keyword not only in Lisp and its descendants, but also in Python.

Guido van Rossum, the creator of Python, wrote a blog post about the design principle that led not only to first-class functions in Python, but first-class methods as well: [First-class Everything](http://python-history.blogspot.com/2009/02/first-class-everything.html).



In [13]:
class MyClass {
    private privateField: number;
    public publicField: number;

    constructor() {
        this.privateField = 1;
        this.publicField = 2;
    }

    private privateMethod() {
        console.log("Private method called.");
    }

    public publicMethod() {
        console.log("Public method called.");
    }
}

let A = new MyClass();
console.log(A.publicField);
// console.log(A.privateField);

21:15 - Property 'privateField' is private and only accessible within class 'MyClass'.


### Function expressions in TypeScript
We have already been using function expressions, starting as far back as Testing, since the mocha framework uses them:


```ts
describe("Math.max", function() {
  it("covers a < b", function() {
    assert.strictEqual(Math.max(1, 2), 2);
  });
});
```
TypeScript also has a more compact arrow syntax that avoids the need for a function keyword:

```ts
describe("Math.max", () => {
  it("covers a < b", () => {
    assert.strictEqual(Math.max(1, 2), 2);
  });
});
```
If the body of the function consists of only a single expression (i.e. like a Python lambda expression), then even the curly braces can be omitted:

```ts
it("covers a < b", () => assert.strictEqual(Math.max(1, 2), 2) );
```
But it turns out that there is a technical difference between [arrow functions](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Functions/Arrow_functions) and function expressions that is important when using methods: a function expression may redefine **this**, but an arrow function uses the definition of **this** from its surrounding context. So arrow functions should always be used inside an instance method, rather than function expressions. Understanding This, [Bind, Call, and Apply](https://www.digitalocean.com/community/conceptual_articles/understanding-this-bind-call-and-apply-in-javascript) in JavaScript has a good explanation of the issues behind the meaning of this in JavaScript.

## Abstracting out control flow

In this reading we discuss map/filter/reduce, a design pattern that substantially simplifies the implementation of functions that operate over sequences of elements. In this example, we’ll have lots of sequences — arrays of files; input streams that are sequences of lines; lines that are sequences of words. Map/filter/reduce will enable us to operate on those sequences with no explicit control flow — not a single for loop or if statement.

Suppose we’re given the following problem: write a function that finds all the words in the TypeScript files in your project.

Following good practice, we break it down into several simpler steps and write a function for each one:

* find all the files in the project, by scanning recursively from the project’s root folder
* restrict them to files with a particular suffix, in this case .ts
* open each file and read it in line-by-line
* break each line into words


Writing the individual functions for these substeps, we’ll find ourselves writing a lot of low-level iteration code. For example, here’s what the recursive traversal of the project folder might look like:

In [16]:
import fs from 'fs';
import path from 'path';

/**
 * Find names of all files in the filesystem subtree rooted at folder.
 * @param folder root of subtree, requires fs.lstatSync(folder).isDirectory() === true
 * @returns array of names of all ordinary files (not folders) that have folder as
 *          their ancestor
 */
function allFilesIn(folder: string): Array<string> {
    let files: Array<string> = [];
    for (const child of fs.readdirSync(folder)) {
        const fullNameOfChild = path.join(folder, child);
        if (fs.lstatSync(fullNameOfChild).isDirectory()) {
            files = files.concat(allFilesIn(fullNameOfChild));
        } else if (fs.lstatSync(fullNameOfChild).isFile()) {
            files.push(fullNameOfChild);
        }
    }
    return files;
}

And here’s what the filtering function might look like, which restricts that file array down to just the TypeScript files. Imagine calling this like onlyFilesWithSuffix(files, ".ts"):



In [ ]:
/**
 * Filter an array of files to those that end with suffix.
 * @param filenames array of filenames
 * @param suffix string to test
 * @returns a new array consisting of only those files whose names end with suffix
 */
function onlyFilesWithSuffix(filenames: Array<string>, suffix: string): Array<string> {
    const result: Array<string> = [];
    for (const f of filenames) {
        if (f.endsWith(suffix)) {
            result.push(f);
        }
    }
    return result;
}

[→ full code for the example](https://web.mit.edu/6.031/www/sp23/classes/13-functional/code.html#words1ts)

### Iterator abstraction

One design pattern that abstracts away from the details of iterating over a data structure is an iterator. An iterator gives you a sequence of elements from a data structure, without you having to worry about whether the data structure is a set or a hash table or an array — the Iterator looks the same no matter what the data structure is.

For example, given an Array<string> filenames, we can iterate using indices:

```ts
for (let i = 0; i < filenames.length; i++) {
    let f = filenames[i];
    // ...
```

But this code depends on the length property of Array, as well as [] for indexing, which might be different in another data structure. (A Set, for example, uses size instead of length, and has no [] indexing at all.)

Using an iterator abstracts away the details:

```ts
const iter: Iterator<string> = filenames.iterator();
while (true) {
    let next = iter.next();
    if (next.done) break;
    let f = next.value;
    // ...
```
Now the loop will be identical for any iterable type that provides an Iterator. filenames could be an Array or Set or any other iterable type, and the code would work the same. In TypeScript, these types all provide the [iterator](https://www.typescriptlang.org/docs/handbook/iterators-and-generators.html#forof-statements) method. Any such iterable type can be used with TypeScript’s [for..of](https://www.typescriptlang.org/docs/handbook/iterators-and-generators.html#forof-statements) statement — here, for (let f of filenames) — and under the hood, it uses an iterator.

#### Map/filter/reduce abstraction

The map/filter/reduce patterns in this reading do something similar to Iterator, but at an even higher level: they treat the entire sequence of elements as a unit. In this paradigm, the control statements disappear: specifically, the for statements, the if statements, and the return statements in the code from our introductory example will be gone. We’ll also be able to get rid of most of the temporary names (i.e., the local variables filenames, f, and result).

We’ll have three operations for iterable types: map, filter, and reduce. Let’s look at each one in turn, and then look at how they work together. We’ll focus on using Array as the iterable type, because it provides map, filter, and reduce operations natively in TypeScript.



## Map

Map applies a [unary ](https://en.wikipedia.org/wiki/Unary_function)function to each element in the sequence and returns a new sequence containing the results, in the same order:

map : Array<‍E> × (E → F) → Array<‍F>
For example:

```ts
const array: Array<number> = [1, 4, 9, 16];
const result = array.map(Math.sqrt);
```
This code starts with an array containing the numbers 1, 4, 9, 16 and applies the square root function to each element to produce the values 1.0, 2.0, 3.0, 4.0.

In the call array.map(Math.sqrt):

* map has type Array<‍number> × (number → number) → Array<‍number>
* the input array has type Array<‍number>
* the mapping function Math.sqrt has type number → number
* the return value has type Array<‍number>

We can write the same expression more compactly as:
```ts
[1, 4, 9, 16].map(Math.sqrt);

```
which is the syntax that future examples will use.

Another example of a map:


```ts
["A", "b", "C"].map(s => s.toLocaleLowerCase())
```
which produces an array containing "a", "b", "c". (Notice that when we call a function like Math.sqrt, we can refer to it directly. Calling a method like toLocaleLowerCase requires a lambda expression to wrap the method call syntax.)

Map is useful even if you don’t care about the return value of the function. When you have a sequence of mutable objects, for example, you may want to map a mutator operation over them. Because mutator operations typically return void, however, you can use [forEach ](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/forEach)instead of map. forEach applies the function to each element of the sequence, but does not collect their return values into a new sequence:
```ts
itemsToRemove.forEach(item => mySet.delete(item));
```
(A lambda is needed here, too. The expression mySet.delete by itself would refer to the delete function without binding mySet to this, which will not work.)

The **map** and **forEach** operations capture a common pattern for operating over sequences: doing the same thing to each element of the sequence.







## Filter

Our next important sequence operation is filter, which tests each element of an Array<E> with a unary function from E to boolean.

Elements that satisfy the [predicate](https://web.mit.edu/6.031/www/sp23/classes/05-designing-specs/#^predicate) are kept; those that don’t are removed. A new sequence is returned; filter doesn’t modify its input sequence.

filter : Array<‍E> × (E → boolean) → Array<‍E>

## Reduce

Our final operator, reduce, combines the elements of the sequence together, using a binary function. In addition to the function and the array, it also takes an initial value that initializes the reduction, and that ends up being the return value if the array is empty:

reduce : Array<‍E> × (E × E → E) × E → E

arr.reduce(f, init) combines the elements of the array together. Here is one way that it might compute the result:

>result0 = init
result1 = f(result0, arr[0])
result2 = f(result1, arr[1])
...
$resultn = f(resultn-1, arr[n-1])
resultn is the final result for an n-element sequence.

Adding numbers is probably the most straightforward example:
```ts
[1,2,3]
    .reduce((x,y) => x+y, 0)
// computes (((0+1)+2)+3) to produce 6
```


### Initial value
There are three design choices in the reduce operation. First is whether to require an initial value. In TypeScript, the initial value can be omitted, in which case reduce uses the first element of the sequence as the initial value of the reduction. But if the sequence is empty, then reduce has no value to return, and reduce throws a TypeError. It’s important to keep this in mind when using reducers like max, which have no well-defined initial value:

```ts
[5, 8, 3, 1].reduce((x,y) => Math.max(x,y))
// computes max(max(max(5,8),3),1) and returns 8

[].reduce((x,y) => Math.max(x,y))
// throws TypeError!
```

###  Reduction to another type

The second design choice is the return type of the reduce operation. It doesn’t necessarily have to match the element type of the original sequence. For example, we can use reduce to concatenate an array of numbers (type E) into a string (type F). This changes the reduce operation in two ways:

* the initial value now has type F
* the binary function is now an accumulator, of type F × E → F, that . takes the current result (of type F) and a new element from the sequence (of type E), and produces an accumulated result of type F.

So a more general form of the reduce operation now looks like this:

reduce : Array<‍E> × (F × E → F) × F → F
>
    Array<‍E>：这表示一个泛型数组类型，其中 <‍E> 表示数组中元素的类型。这个数组可以包含类型为 E 的元素。

    (F × E → F)：这是一个函数类型，接受两个参数 F 和 E，并返回一个类型为 F 的结果。这里 × 表示函数的参数和返回值之间的映射关系。

    F：这是一个类型，表示某种数据类型。

    F → F：这也是一个函数类型，接受一个参数 F 并返回一个类型为 F 的结果。

整个表达式可以理解为一个函数类型，其输入是一个数组（Array<‍E>），一个函数（F × E → F），和一个类型为 F 的参数。函数的返回值是一个类型为 F 的结果。

这种类型签名的表达方式在函数式编程中很常见，它用于描述函数的输入和输出类型，以便在代码中使用和组合不同的函数。

In the special case where F is the same type as E, this type signature is the same as above.

Here’s a simple example that concatenates a sequence of numbers into a string:

```ts
[1,2,3].reduce( (s: string, n: number) => s + n, "" );
// returns "123"
```

We’ve included parameter type declarations in the lambda expression above in order to clarify the difference between the two parameters of the accumulator function. If **F** is the same type as **E**, it’s ok to omit these type declarations in the lambda expression.

<p id="@third_design_choice"><a class="jump" href="#@third_design_choice"></a>The third design choice is the order in which the elements are accumulated.
TypeScript’s <code>reduce</code> operation combines the sequence starting from the left (the first element):</p>

<blockquote class="no-markdown">result<sub>0</sub> = init
<br>result<sub>1</sub> = f(result<sub>0</sub>, arr[0])
<br>result<sub>2</sub> = f(result<sub>1</sub>, arr[1])
<br>...
<br>result<sub>n</sub> = f(result<sub>n-1</sub>, arr[n-1])
</blockquote>

<p id="@another_typescript_operation"><a class="jump" href="#@another_typescript_operation"></a>Another TypeScript operation, <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/reduceRight"><code>reduceRight</code></a>, goes in the other direction:</p>

<blockquote class="no-markdown">result<sub>0</sub> = init
<br>result<sub>1</sub> = f(result<sub>0</sub>, arr[n-1])
<br>result<sub>2</sub> = f(result<sub>1</sub>, arr[n-2])
<br>...
<br>result<sub>n</sub> = f(result<sub>n-1</sub>, arr[0])
</blockquote>

<p class="no-markdown" id="@produce_resultn_final"><a class="jump" href="#@produce_resultn_final"></a>to produce result<sub>n</sub> as the final result.</p>

<p id="@here-s_diagram_two"><a class="jump" href="#@here-s_diagram_two"></a>Here’s a diagram of the two ways to reduce, from the left or from the right.
If the operator is non-commutative (like string concatenation, shown here), then each direction might produce a different answer:</p>

<table class="no-markdown" id="@reduce_1_2">
<tbody><tr>
<td rowspan="2"><a class="jump" href="#@reduce_1_2"></a><img src="ref/lect13/2023-08-14-16-33-30.png"></td>
<td valign="top" style="text-align: left; padding: .75em 1.5em .75em 1.5em;"><strong>reduce( [1, 2, 3], + , "" )</strong><br> = (("" + 1) + 2) + 3<br> = "123"</td>
<td style="padding: 2em;"></td>
<td valign="top" style="text-align: right; padding: .75em 1.5em .75em 1.5em;"><strong>reduceRight( [1, 2, 3], + , "" )</strong><br> = (("" + 3) + 2) + 1<br> = "321"</td>
<td rowspan="2"><img src="ref/lect13/2023-08-14-16-31-42.png"></td>
</tr>
<tr>

</tbody></table>

## Back to the motivating example

<div data-outline="back_to_the_motivating_example">

<p id="@going_back_example"><a class="jump" href="#@going_back_example"></a>Going back to the example we started with, where we want to find all the words in the TypeScript files in our project, let’s try creating a useful abstraction for filtering files by suffix:</p>

<pre id="@const_endswith_suffix"><a class="jump" href="#@const_endswith_suffix"></a><code class="language-typescript hljs"><span class="hljs-keyword">const</span> endsWith = <span class="hljs-function">(<span class="hljs-params">suffix: <span class="hljs-built_in">string</span></span>) =&gt;</span> {
    <span class="hljs-keyword">return</span> <span class="hljs-function">(<span class="hljs-params">filename: <span class="hljs-built_in">string</span></span>) =&gt;</span> filename.endsWith(suffix);
}</code></pre>

<p id="@typescript-s_string-endswith_function"><a class="jump" href="#@typescript-s_string-endswith_function"></a>TypeScript’s <code>string.endsWith</code> is a function <strong><code>string × string → boolean</code></strong>.</p>

<p id="@our_new_endswith"><a class="jump" href="#@our_new_endswith"></a>Our new <code>endsWith</code> wrapper returns <em>functions</em> that are useful as filters.
It takes a filename suffix like <code>.ts</code> and dynamically generates a function that we can use with filter to test for that suffix.
Given a <code>Array&lt;string&gt; filenames</code>, we can now write, e.g., <code>filenames.filter(endsWith(".ts"))</code> to obtain a new filtered array.</p>

<p id="@endswith_different_kind"><a class="jump" href="#@endswith_different_kind"></a><code>endsWith</code> is a different kind of beast than our usual functions.
It’s a <mark data-structure-text="higher-order function" id="^higher-order_function"><strong>higher-order function</strong></mark>, meaning that it’s a function that takes another function as an argument, or returns another function as its result, as <code>endsWith</code> does.
Higher-order functions are operations on the data type of functions.
In this case, <code>endsWith</code> is a <em>creator</em> of functions.</p>

<p id="@signature_string_string"><a class="jump" href="#@signature_string_string"></a>Its signature is: <strong><code>string → (string → boolean</code>)</strong>.</p>

<p id="@now_use_map"><a class="jump" href="#@now_use_map"></a>Now let’s use map and filter to recursively traverse the folder tree:</p>

<div class="pull-margin"><pre id="@function_allfilesin-folder_string"><a class="jump" href="#@function_allfilesin-folder_string"></a><code class="language-typescript hljs"><span class="hljs-function"><span class="hljs-keyword">function</span> <span class="hljs-title">allFilesIn</span>(<span class="hljs-params">folder: <span class="hljs-built_in">string</span></span>): <span class="hljs-title">Array</span>&lt;<span class="hljs-title">string</span>&gt; </span>{
    <span class="hljs-keyword">const</span> children: <span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">string</span>&gt; = fs.readdirSync(folder)
                                    .map(<span class="hljs-function"><span class="hljs-params">f</span> =&gt;</span> path.join(folder, f));
    <span class="hljs-keyword">const</span> descendants: <span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">string</span>&gt; = children
                                       .filter(<span class="hljs-function"><span class="hljs-params">f</span> =&gt;</span> fs.lstatSync(f).isDirectory())
                                       .map(allFilesIn)
                                       .flat();
    <span class="hljs-keyword">return</span> [
        ...descendants,
        ...children.filter(<span class="hljs-function"><span class="hljs-params">f</span> =&gt;</span> fs.lstatSync(f).isFile())
    ];
}</code></pre></div>

<p id="@first_line_gets"><a class="jump" href="#@first_line_gets"></a>The first line gets all the children of the folder, which might look like this:</p>

<pre id="@src-client_src-server_src-main-ts"><a class="jump" href="#@src-client_src-server_src-main-ts"></a><code class="hljs json">[<span class="hljs-string">"src/client"</span>, <span class="hljs-string">"src/server"</span>, <span class="hljs-string">"src/Main.ts"</span>, ...]
</code></pre>

<p id="@second_line_key"><a class="jump" href="#@second_line_key"></a>The second line is the key bit: it filters the children for just the subfolders using the <code>isDirectory</code> method, and then recursively maps <code>allFilesIn</code> against this array of subfolders!
The result might look like this:</p>

<pre><code class="hljs json">[[<span class="hljs-string">"src/client/MyClient.ts"</span>, ...], [<span class="hljs-string">"src/server/MyServer.ts"</span>, ...], ...]
</code></pre>

<p id="@so_flatten_remove"><a class="jump" href="#@so_flatten_remove"></a>So we have to <em>flatten</em> it to remove the nested structure, which is what <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/flat"><code>flat</code></a>) does:</p>

<pre><code class="hljs json">[<span class="hljs-string">"src/client/MyClient.ts"</span>, ..., <span class="hljs-string">"src/server/MyServer.ts"</span>, ...]
</code></pre>

<p id="@pattern_map-and-flatten_common"><a class="jump" href="#@pattern_map-and-flatten_common"></a>(This pattern of map-and-flatten is common enough that arrays also have a method to do both that we could have used instead: <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/flatMap"><code>flatMap</code></a>.)</p>

<p id="@finally_add_immediate"><a class="jump" href="#@finally_add_immediate"></a>Finally we add the immediate children that are plain files (not folders), and that’s our result.</p>

<p id="@can_also_do"><a class="jump" href="#@can_also_do"></a>We can also do the other pieces of the problem with map/filter/reduce.
Once we have the array of all files underneath the current folder, we can filter it to just the TS files:</p>

<pre id="@const_filenames_array-string"><a class="jump" href="#@const_filenames_array-string"></a><code class="language-typescript hljs"><span class="hljs-keyword">const</span> filenames: <span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">string</span>&gt; = allFilesIn(<span class="hljs-string">"."</span>)
                                 .filter(endsWith(<span class="hljs-string">".ts"</span>));</code></pre>

<p id="@now_files_want"><a class="jump" href="#@now_files_want"></a>Now that we have the files we want to extract words from, we’re ready to load their contents:</p>

<pre id="@const_filecontents_array-array-string"><a class="jump" href="#@const_filecontents_array-array-string"></a><code class="language-typescript hljs"><span class="hljs-keyword">const</span> fileContents: <span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">string</span>&gt;&gt; = filenames.map(<span class="hljs-function"><span class="hljs-params">f</span> =&gt;</span> {
    <span class="hljs-keyword">try</span> {
        <span class="hljs-keyword">const</span> data = fs.readFileSync(f, { encoding: <span class="hljs-string">"utf8"</span>, flag: <span class="hljs-string">"r"</span> });
        <span class="hljs-keyword">return</span> data.split(<span class="hljs-string">"\n"</span>);
    } <span class="hljs-keyword">catch</span> (e) {
        <span class="hljs-built_in">console</span>.error(e);
    }
});</code></pre>

<p id="@finally_can_flatten"><a class="jump" href="#@finally_can_flatten"></a>Finally, we can flatten the array of arrays of lines into a simple array of lines:</p>

<pre id="@const_lines_array-string"><a class="jump" href="#@const_lines_array-string"></a><code class="language-typescript hljs"><span class="hljs-keyword">const</span> lines: <span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">string</span>&gt; = fileContents.flat();</code></pre>

<p id="@then_extract_nonempty"><a class="jump" href="#@then_extract_nonempty"></a>and then extract the nonempty words from each line:</p>

<pre id="@const_words_array-string"><a class="jump" href="#@const_words_array-string"></a><code class="language-typescript hljs"><span class="hljs-keyword">const</span> words: <span class="hljs-built_in">Array</span>&lt;<span class="hljs-built_in">string</span>&gt; = lines.map(<span class="hljs-function"><span class="hljs-params">line</span> =&gt;</span> line.split(<span class="hljs-regexp">/\W+/</span>)
                                               .filter(<span class="hljs-function"><span class="hljs-params">s</span> =&gt;</span> s.length &gt; <span class="hljs-number">0</span>))
                             .flat();</code></pre>

<p id="@here_call_split"><a class="jump" href="#@here_call_split"></a>(Here we call <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String/split"><code>split</code></a> not with a simple string delimiter, but with a <em><a href="../12-regex-grammars/#regular_expressions">regular expression</a></em>, which we discussed in a previous reading.
This regular expression identifies substrings of non-word characters.)</p>

<p id="@done_our_array"><a class="jump" href="#@done_our_array"></a>And we’re done, we have our array of all words in the project’s TypeScript files!
As promised, the control statements have disappeared.</p>


</div>

## Benefits of abstracting out control

Map/filter/reduce can often make code shorter and simpler, and allow the programmer to focus on the heart of the computation rather than on the details of loops, branches, and control flow.

### Avoiding loops in TypeScript

<div data-outline="avoiding_loops_in_typescript">

<p id="@python_makes_filter"><a class="jump" href="#@python_makes_filter"></a>Python makes <code>filter</code> and <code>map</code> easily accessible with list comprehensions:</p>

<pre id="@doubleodds_x-2_x"><a class="jump" href="#@doubleodds_x-2_x"></a><code class="language-python hljs">doubleOdds = [ x*<span class="hljs-number">2</span> <span class="hljs-keyword">for</span> x <span class="hljs-keyword">in</span> arr <span class="hljs-keyword">if</span> x % <span class="hljs-number">2</span> == <span class="hljs-number">1</span> ]</code></pre>

<p id="@which_like_typescript"><a class="jump" href="#@which_like_typescript"></a>… which is like the TypeScript:</p>

<pre id="@const_doubleodds_arr-filter-x"><a class="jump" href="#@const_doubleodds_arr-filter-x"></a><code class="language-ts hljs typescript"><span class="hljs-keyword">const</span> doubleOdds = arr.filter(<span class="hljs-function"><span class="hljs-params">x</span> =&gt;</span> x % <span class="hljs-number">2</span> === <span class="hljs-number">1</span>).map(<span class="hljs-function"><span class="hljs-params">x</span> =&gt;</span> x*<span class="hljs-number">2</span>)</code></pre>

<p id="@result_list_comprehension"><a class="jump" href="#@result_list_comprehension"></a>The result of a list comprehension is a list by virtue of the <code>[</code>…<code>]</code> brackets.
In Python, comprehensions work over any iterable source, and can also be used to build sets and dictionaries.
For example:</p>

<pre id="@arr_4_7"><a class="jump" href="#@arr_4_7"></a><code class="language-python hljs">arr = [ <span class="hljs-number">4</span>, <span class="hljs-number">7</span>, <span class="hljs-number">5</span>, <span class="hljs-number">6</span>, <span class="hljs-number">7</span>, <span class="hljs-number">4</span> ]  <span class="hljs-comment"># given a list</span>
{ x <span class="hljs-keyword">for</span> x <span class="hljs-keyword">in</span> arr <span class="hljs-keyword">if</span> x &gt; <span class="hljs-number">5</span> } <span class="hljs-comment"># ... create a set of values greater than 5:</span>
<span class="hljs-comment"># =&gt; a set { 6, 7 }</span>

input = { <span class="hljs-string">'apple'</span>: <span class="hljs-string">'red'</span>, <span class="hljs-string">'banana'</span>: <span class="hljs-string">'yellow'</span> } <span class="hljs-comment"># given a dictionary</span>
{ value: key <span class="hljs-keyword">for</span> key, value <span class="hljs-keyword">in</span> input.items() } <span class="hljs-comment"># ... create a dictionary with</span>
                                               <span class="hljs-comment">#     key-value pairs inverted</span>
<span class="hljs-comment"># =&gt; a dictionary { 'red': 'apple', 'yellow': 'banana' }</span></code></pre>

<p id="@can_do_same"><a class="jump" href="#@can_do_same"></a>We can do the same things in TypeScript:</p>

<pre id="@const_arr_4"><a class="jump" href="#@const_arr_4"></a><code class="language-ts hljs typescript"><span class="hljs-keyword">const</span> arr = [ <span class="hljs-number">4</span>, <span class="hljs-number">7</span>, <span class="hljs-number">5</span>, <span class="hljs-number">6</span>, <span class="hljs-number">7</span>, <span class="hljs-number">4</span> ];
<span class="hljs-keyword">new</span> Set(arr.filter(<span class="hljs-function"><span class="hljs-params">x</span> =&gt;</span> x &gt; <span class="hljs-number">5</span>));
<span class="hljs-comment">// =&gt; a Set { 6, 7 }</span>

<span class="hljs-keyword">const</span> input = <span class="hljs-keyword">new</span> Map([ [<span class="hljs-string">'apple'</span>,<span class="hljs-string">'red'</span>], [<span class="hljs-string">'banana'</span>,<span class="hljs-string">'yellow'</span>] ]);
<span class="hljs-keyword">new</span> Map([ ...input.entries() ].map(<span class="hljs-function"><span class="hljs-params">entry</span> =&gt;</span> entry.reverse()));
<span class="hljs-comment">// =&gt; a Map { 'red' =&gt; 'apple', 'yellow' =&gt; 'banana' }</span></code></pre>

<p id="@breaking_down_construction"><a class="jump" href="#@breaking_down_construction"></a>Breaking down the construction of that inverted map:</p>

<ul id="@notice_map_constructor">
<li><a class="jump" href="#@notice_map_constructor"></a>Notice that the <code>Map</code> constructor takes a series of two-element arrays: the first element of each is a key, the second is its value.</li>
<li><a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Map/entries"><code>Map.entries</code></a> returns an iterable that iterates over two-element key-value arrays.</li>
<li>Because JavaScript/TypeScript doesn’t provide <code>map</code> and <code>filter</code> on iterables, only on arrays, we use the <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Spread_syntax">spread operator</a> <code>...</code> to stuff those key-value arrays into an array.</li>
<li>Now we can call <code>map</code> on that array, and for each <code>[key,value]</code> entry, we reverse it to <code>[value,key]</code> and create the new map.</li>
</ul>

<p id="@just_idiomatic_python"><a class="jump" href="#@just_idiomatic_python"></a>Just as idiomatic Python code will avoid <code>for</code> loops in favor of comprehensions for filtering and mapping data structures, idiomatic TypeScript will take advantage of <code>map</code> and <code>filter</code> in concert with tools like <code>Map.entries</code> and array methods like <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/entries"><code>Array.entries</code></a>, <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/some"><code>.some</code></a>, and <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/every"><code>.every</code></a> to avoid loops.</p>

<p id="@any_time_you"><a class="jump" href="#@any_time_you"></a>Any time you write a loop, reach for those tools first, instead of an index counter.</p>

</div>

### Making an abstract syntax tree

<div data-outline="making_an_abstract_syntax_tree">

<p id="@you_may_remember"><a class="jump" href="#@you_may_remember"></a>You may remember <a href="../12-regex-grammars/#makeAbstractSyntaxTree"><code>makeAbstractSyntaxTree()</code></a> from a previous reading, which walks over a parse tree to create an abstract syntax tree.
Here is a snippet of its original code, using a <code>for</code> loop: </p>

<pre id="@else_if_parsetree-name_integergrammar-sum_sum_primary_primary_const_children_array-parsetree-integergrammar_parsetree-childrenbyname-integergrammar-primary_let"><a class="jump" href="#@else_if_parsetree-name_integergrammar-sum_sum_primary_primary_const_children_array-parsetree-integergrammar_parsetree-childrenbyname-integergrammar-primary_let"></a><code class="language-ts hljs typescript">...
} <span class="hljs-keyword">else</span> <span class="hljs-keyword">if</span> (parseTree.name === IntegerGrammar.Sum) {
    <span class="hljs-comment">// sum ::= primary ('+' primary)*;</span>
    <span class="hljs-keyword">const</span> children: <span class="hljs-built_in">Array</span>&lt;ParseTree&lt;IntegerGrammar&gt;&gt; = parseTree.childrenByName(IntegerGrammar.Primary);
    <span class="hljs-keyword">let</span> expression: IntegerExpression = makeAbstractSyntaxTree(children[<span class="hljs-number">0</span>] ?? assert.fail(<span class="hljs-string">"missing child"</span>));
    <span class="hljs-keyword">for</span> (<span class="hljs-keyword">let</span> i = <span class="hljs-number">1</span>; i &lt; children.length; ++i) {
        expression = <span class="hljs-keyword">new</span> Plus(expression, makeAbstractSyntaxTree(children[i] ?? assert.fail(<span class="hljs-string">"missing child"</span>)));
    }
    <span class="hljs-keyword">return</span> expression;
} ...</code></pre>

<p id="@simplify_code_using"><a class="jump" href="#@simplify_code_using"></a>Let’s simplify this code using map/filter/reduce.</p>

<div class="reading-exercises exercises panel-group converted" id="ex-benefits_of_abstracting_out_control_2"><iframe class="exercises-status" src="https://6031.mit.edu/handx/sp23/status.php"></iframe><h4 class="text-danger">reading exercises</h4><div class="panel panel-danger"><div class="panel-heading" data-structure-tag="exercise" id="@ex-benefits_of_abstracting_out_control_2-parsing_with_mapfilterreduce" data-target="#ex-benefits_of_abstracting_out_control_2-parsing_with_mapfilterreduce" data-toggle="collapse"><a class="jump" href="#@ex-benefits_of_abstracting_out_control_2-parsing_with_mapfilterreduce"></a><span class="panel-title">Parsing with map/filter/reduce</span></div><div class="panel-collapse collapse exercise-panel" id="ex-benefits_of_abstracting_out_control_2-parsing_with_mapfilterreduce" data-outline="parsing_with_mapfilterreduce" data-ex-id="making_an_abstract_syntax_tree/parsing_with_mapfilterreduce" data-ex-category="reading-exercises" data-ex-remote="https://6031.mit.edu/handx/sp23/submit.php" data-ex-handout="classes-13-functional"><div class="panel-body">

<p id="@which_these_operations"><a class="jump" href="#@which_these_operations"></a>Which of these operations are appropriate for simplifying the <code>for</code> loop above?</p>

<div class="form-group exercise-part" data-outline="a"><div class="checkbox exercise-choice" data-outline="map"><label for="md_converted_choice_17_0"><input type="checkbox" id="md_converted_choice_17_0">map</label><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
<div class="checkbox exercise-choice" data-outline="filter"><label for="md_converted_choice_17_1"><input type="checkbox" id="md_converted_choice_17_1">filter</label><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
<div class="checkbox exercise-choice" data-outline="reduce"><label for="md_converted_choice_17_2"><input type="checkbox" id="md_converted_choice_17_2">reduce</label><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
</div>

<div class="exercise-explain exercise-remote"><p>(missing explanation)</p></div>

<div class="form-inline"><div class="form-group"><button class="btn btn-default exercise-submit">check</button> <button class="btn btn-default exercise-reveal" style="display: none;">explain</button></div><div class="exercise-progress progress"><div class="progress-bar progress-bar-danger progress-bar-striped active"></div></div><div class="exercise-error"></div></div></div></div></div>

<div class="panel panel-danger"><div class="panel-heading" data-structure-tag="exercise" id="@ex-benefits_of_abstracting_out_control_2-parsing_part_1" data-target="#ex-benefits_of_abstracting_out_control_2-parsing_part_1" data-toggle="collapse"><a class="jump" href="#@ex-benefits_of_abstracting_out_control_2-parsing_part_1"></a><span class="panel-title">Parsing part 1</span></div><div class="panel-collapse collapse exercise-panel" id="ex-benefits_of_abstracting_out_control_2-parsing_part_1" data-outline="parsing_part_1" data-ex-id="making_an_abstract_syntax_tree/parsing_part_1" data-ex-category="reading-exercises" data-ex-remote="https://6031.mit.edu/handx/sp23/submit.php" data-ex-handout="classes-13-functional"><div class="panel-body">

<pre id="@else_if_parsetree-name_integergrammar-sum_sum_primary_primary_const_children_array-parsetree-integergrammar_parsetree-childrenbyname-integergrammar-primary_const_subexprs_array-integerexpression_children-map-_______________"><a class="jump" href="#@else_if_parsetree-name_integergrammar-sum_sum_primary_primary_const_children_array-parsetree-integergrammar_parsetree-childrenbyname-integergrammar-primary_const_subexprs_array-integerexpression_children-map-_______________"></a><code class="language-ts hljs typescript">...
} <span class="hljs-keyword">else</span> <span class="hljs-keyword">if</span> (parseTree.name === IntegerGrammar.Sum) {
    <span class="hljs-comment">// sum ::= primary ('+' primary)*;</span>
    <span class="hljs-keyword">const</span> children: <span class="hljs-built_in">Array</span>&lt;ParseTree&lt;IntegerGrammar&gt;&gt; = parseTree.childrenByName(IntegerGrammar.Primary);

    <span class="hljs-keyword">const</span> subexprs: <span class="hljs-built_in">Array</span>&lt;IntegerExpression&gt; = children.map(_______________);

    <span class="hljs-keyword">const</span> expression: IntegerExpression = subexprs.reduce(...);

    <span class="hljs-keyword">return</span> expression;
} ...</code></pre>

<p id="@can_fill_blank_map_call_above"><a class="jump" href="#@can_fill_blank_map_call_above"></a>What can fill the blank of the <code>map</code> call above? Use just one identifier, rather than a lambda expression.</p>

<div class="form-group exercise-part" data-outline="a"><div class="textfield exercise-choice"><input type="text" class="form-control" style="width: 37%;"><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
</div>

<div class="exercise-explain exercise-remote"><p>(missing explanation)</p></div>

<div class="form-inline"><div class="form-group"><button class="btn btn-default exercise-submit">check</button> <button class="btn btn-default exercise-reveal" style="display: none;">explain</button></div><div class="exercise-progress progress"><div class="progress-bar progress-bar-danger progress-bar-striped active"></div></div><div class="exercise-error"></div></div></div></div></div>

<div class="panel panel-danger"><div class="panel-heading" data-structure-tag="exercise" id="@ex-benefits_of_abstracting_out_control_2-parsing_part_2" data-target="#ex-benefits_of_abstracting_out_control_2-parsing_part_2" data-toggle="collapse"><a class="jump" href="#@ex-benefits_of_abstracting_out_control_2-parsing_part_2"></a><span class="panel-title">Parsing part 2</span></div><div class="panel-collapse collapse exercise-panel" id="ex-benefits_of_abstracting_out_control_2-parsing_part_2" data-outline="parsing_part_2" data-ex-id="making_an_abstract_syntax_tree/parsing_part_2" data-ex-category="reading-exercises" data-ex-remote="https://6031.mit.edu/handx/sp23/submit.php" data-ex-handout="classes-13-functional"><div class="panel-body">

<pre id="@else_if_parsetree-name_integergrammar-sum_sum_primary_primary_const_children_array-parsetree-integergrammar_parsetree-childrenbyname-integergrammar-primary_const_subexprs_array-integerexpression_children-map"><a class="jump" href="#@else_if_parsetree-name_integergrammar-sum_sum_primary_primary_const_children_array-parsetree-integergrammar_parsetree-childrenbyname-integergrammar-primary_const_subexprs_array-integerexpression_children-map"></a><code class="language-ts hljs typescript">...
} <span class="hljs-keyword">else</span> <span class="hljs-keyword">if</span> (parseTree.name === IntegerGrammar.Sum) {
    <span class="hljs-comment">// sum ::= primary ('+' primary)*;</span>
    <span class="hljs-keyword">const</span> children: <span class="hljs-built_in">Array</span>&lt;ParseTree&lt;IntegerGrammar&gt;&gt; = parseTree.childrenByName(IntegerGrammar.Primary);

    <span class="hljs-keyword">const</span> subexprs: <span class="hljs-built_in">Array</span>&lt;IntegerExpression&gt; = children.map(...);

    <span class="hljs-keyword">const</span> expression: IntegerExpression = subexprs.reduce(<span class="hljs-function">(<span class="hljs-params">result, subexpr</span>) =&gt;</span> ___________);

    <span class="hljs-keyword">return</span> expression;
} ...</code></pre>

<p id="@can_fill_blank_reduce_call_above"><a class="jump" href="#@can_fill_blank_reduce_call_above"></a>What can fill the blank of the <code>reduce</code> call above, to produce the final abstract syntax tree node representing this sum?</p>

<div class="form-group exercise-part" data-outline="a"><div class="textfield exercise-choice"><input type="text" class="form-control" style="width: 50%;"><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
</div>

<div class="exercise-explain exercise-remote"><p>(missing explanation)</p></div>

<div class="form-inline"><div class="form-group"><button class="btn btn-default exercise-submit">check</button> <button class="btn btn-default exercise-reveal" style="display: none;">explain</button></div><div class="exercise-progress progress"><div class="progress-bar progress-bar-danger progress-bar-striped active"></div></div><div class="exercise-error"></div></div></div></div></div>

<div class="panel panel-danger"><div class="panel-heading" data-structure-tag="exercise" id="@ex-benefits_of_abstracting_out_control_2-what_to_assert" data-target="#ex-benefits_of_abstracting_out_control_2-what_to_assert" data-toggle="collapse"><a class="jump" href="#@ex-benefits_of_abstracting_out_control_2-what_to_assert"></a><span class="panel-title">What to assert</span></div><div class="panel-collapse collapse exercise-panel" id="ex-benefits_of_abstracting_out_control_2-what_to_assert" data-outline="what_to_assert" data-ex-id="making_an_abstract_syntax_tree/what_to_assert" data-ex-category="reading-exercises" data-ex-remote="https://6031.mit.edu/handx/sp23/submit.php" data-ex-handout="classes-13-functional"><div class="panel-body">

<p id="@original_loop_code"><a class="jump" href="#@original_loop_code"></a>The original <code>for</code> loop code had two assertions in it.
Which of the assertions do we still need (in some form, perhaps written differently) in our new map/reduce code?</p>

<div class="form-group exercise-part" data-outline="a"><div class="checkbox exercise-choice" data-outline="children[0] ?? assert.fail(&quot;missing child&quot;)"><label for="md_converted_choice_18_0"><input type="checkbox" id="md_converted_choice_18_0"><code>children[0] ?? assert.fail("missing child")</code></label><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
<div class="checkbox exercise-choice" data-outline="children[i] ?? assert.fail(&quot;missing child&quot;)"><label for="md_converted_choice_18_1"><input type="checkbox" id="md_converted_choice_18_1"><code>children[i] ?? assert.fail("missing child")</code></label><span class="exercise-answer exercise-remote" style="display: none;">(missing answer)</span></div>
</div>

<div class="exercise-explain exercise-remote"><p>(missing explanation)</p></div><div class="form-inline"><div class="form-group"><button class="btn btn-default exercise-submit">check</button> <button class="btn btn-default exercise-reveal" style="display: none;">explain</button></div><div class="exercise-progress progress"><div class="progress-bar progress-bar-danger progress-bar-striped active"></div></div><div class="exercise-error"></div></div></div></div></div></div>

</div>

## One more example

<div data-outline="one_more_example">

<p id="@look_typical_database"><a class="jump" href="#@look_typical_database"></a>Let’s look at a typical database query example.
Suppose we have a database about digital cameras, in which each object is of type <code>Camera</code> with various properties (<code>brand</code>, <code>pixels</code>, <code>cost</code>, etc.).
The whole database is in an array called <code>cameras</code>.
Then we can describe queries on this database using map/filter/reduce:</p>

<pre id="@what-s_highest_resolution"><a class="jump" href="#@what-s_highest_resolution"></a><code class="language-typescript hljs"><span class="hljs-comment">// What's the highest resolution Nikon sells? </span>
cameras.filter(<span class="hljs-function"><span class="hljs-params">camera</span> =&gt;</span> camera.brand === <span class="hljs-string">"Nikon"</span>)
       .map(<span class="hljs-function"><span class="hljs-params">camera</span> =&gt;</span> camera.pixels)
       .reduce(<span class="hljs-function">(<span class="hljs-params">x,y</span>) =&gt;</span> <span class="hljs-built_in">Math</span>.max(x,y));</code></pre>

<p id="@relational_databases_use"><a class="jump" href="#@relational_databases_use"></a>Relational databases use the map/filter/reduce paradigm (where it’s called project/select/aggregate).
<a href="https://en.wikipedia.org/wiki/SQL">SQL</a> (Structured Query Language) is the <em>de facto</em> standard language for querying relational databases.
A typical SQL query looks like this:</p>

<pre id="@select_max-pixels_cameras"><a class="jump" href="#@select_max-pixels_cameras"></a><code class="hljs cs"><span class="hljs-function"><span class="hljs-keyword">select</span> <span class="hljs-title">max</span>(<span class="hljs-params">pixels</span>) <span class="hljs-keyword">from</span> cameras <span class="hljs-keyword">where</span> brand</span> = <span class="hljs-string">"Nikon"</span>
</code></pre>

<blockquote>
  <p id="@cameras_sequence_table"><a class="jump" href="#@cameras_sequence_table"></a><code>cameras</code> is a <strong>sequence</strong> (of table rows, where each row has the data for one camera)</p>
  
  <p id="@brand_nikon_filter"><a class="jump" href="#@brand_nikon_filter"></a><code>where brand = "Nikon"</code> is a <strong>filter</strong></p>
  
  <p id="@pixels_map_extracting"><a class="jump" href="#@pixels_map_extracting"></a><code>pixels</code> is a <strong>map</strong> (extracting just the pixels field from the row)</p>
  
  <p id="@max_reduce"><a class="jump" href="#@max_reduce"></a><code>max</code> is a <strong>reduce</strong></p>
</blockquote>

</div><div data-outline="one_more_example">

<p id="@look_typical_database"><a class="jump" href="#@look_typical_database"></a>Let’s look at a typical database query example.
Suppose we have a database about digital cameras, in which each object is of type <code>Camera</code> with various properties (<code>brand</code>, <code>pixels</code>, <code>cost</code>, etc.).
The whole database is in an array called <code>cameras</code>.
Then we can describe queries on this database using map/filter/reduce:</p>

<pre id="@what-s_highest_resolution"><a class="jump" href="#@what-s_highest_resolution"></a><code class="language-typescript hljs"><span class="hljs-comment">// What's the highest resolution Nikon sells? </span>
cameras.filter(<span class="hljs-function"><span class="hljs-params">camera</span> =&gt;</span> camera.brand === <span class="hljs-string">"Nikon"</span>)
       .map(<span class="hljs-function"><span class="hljs-params">camera</span> =&gt;</span> camera.pixels)
       .reduce(<span class="hljs-function">(<span class="hljs-params">x,y</span>) =&gt;</span> <span class="hljs-built_in">Math</span>.max(x,y));</code></pre>

<p id="@relational_databases_use"><a class="jump" href="#@relational_databases_use"></a>Relational databases use the map/filter/reduce paradigm (where it’s called project/select/aggregate).
<a href="https://en.wikipedia.org/wiki/SQL">SQL</a> (Structured Query Language) is the <em>de facto</em> standard language for querying relational databases.
A typical SQL query looks like this:</p>

<pre id="@select_max-pixels_cameras"><a class="jump" href="#@select_max-pixels_cameras"></a><code class="hljs cs"><span class="hljs-function"><span class="hljs-keyword">select</span> <span class="hljs-title">max</span>(<span class="hljs-params">pixels</span>) <span class="hljs-keyword">from</span> cameras <span class="hljs-keyword">where</span> brand</span> = <span class="hljs-string">"Nikon"</span>
</code></pre>

<blockquote>
  <p id="@cameras_sequence_table"><a class="jump" href="#@cameras_sequence_table"></a><code>cameras</code> is a <strong>sequence</strong> (of table rows, where each row has the data for one camera)</p>
  
  <p id="@brand_nikon_filter"><a class="jump" href="#@brand_nikon_filter"></a><code>where brand = "Nikon"</code> is a <strong>filter</strong></p>
  
  <p id="@pixels_map_extracting"><a class="jump" href="#@pixels_map_extracting"></a><code>pixels</code> is a <strong>map</strong> (extracting just the pixels field from the row)</p>
  
  <p id="@max_reduce"><a class="jump" href="#@max_reduce"></a><code>max</code> is a <strong>reduce</strong></p>
</blockquote>

</div>

## Summary

<div data-outline="summary">

<p id="@zooming_out_reading"><a class="jump" href="#@zooming_out_reading"></a>Zooming out, this reading is about <mark data-structure-text="functional programming" id="^functional_programming"><em>functional programming</em></mark>: modeling problems and implementing systems with <em>immutable data</em> and operations that implement <em>pure functions</em>, as opposed to <em>mutable data</em> and operations with <em>side effects</em>.</p>

<p id="@because_functions_typescript"><a class="jump" href="#@because_functions_typescript"></a>Because functions in TypeScript are <em>first-class</em>, it is much easier to write <em>higher-order functions</em> that abstract away control flow code.</p>

<p id="@some_languages_haskell"><a class="jump" href="#@some_languages_haskell"></a>Some languages — <a href="http://www.haskell.org/">Haskell</a>, <a href="http://www.scala-lang.org/">Scala</a>, <a href="http://ocaml.org/">OCaml</a> — are strongly associated with functional programming.
Many other languages — <a href="https://developer.apple.com/swift/">Swift</a>, <a href="http://www.ruby-lang.org/">Ruby</a>, and so on — use functional programming to a greater or lesser extent.</p>

<p id="@functional_programming_helps"><a class="jump" href="#@functional_programming_helps"></a>Functional programming helps make code:</p>

<ul>
<li><p id="@safe_bugs_more"><a class="jump" href="#@safe_bugs_more"></a><strong>Safe from bugs.</strong> 
More immutable data reduces the chance your code has unintended side effects.</p></li>
<li><p id="@easy_understand_chaining"><a class="jump" href="#@easy_understand_chaining"></a><strong>Easy to understand.</strong> 
Chaining operations like <em>map</em>, <em>filter</em>, and <em>reduce</em> in favor of sequential control flow statments like <em>for</em> and <em>if</em> makes your code much easier to read and follow.</p></li>
<li><p id="@ready_change_functional"><a class="jump" href="#@ready_change_functional"></a><strong>Ready for change.</strong> 
The functional style of programming promotes code syntax that follows its semantics, 
which allows the programmer to <a href="#@map-filter-reduce_can_often">focus on the heart of the computation</a> 
and is designed to evolve in tandem with its problem model.</p></li>
</ul></div>

## 